### SETUP INICIAL DO PROJETO

In [1]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# carregar o .env com as credenciais
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")


# detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.sales-salesperson"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"


# configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


#doc: tables_to_process: lista de tabelas que serão processadas
#     output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned

In [2]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

Tabelas a processar: ['desafioadventureworks-446600.raw_data.sales-salesperson']


In [3]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

raw_data
Tabelas disponíveis:
humanresources-employee
person-address
person-businessentity
person-person
person-stateprovince
production-location
production-product
production-productcategory
production-productcosthistory
production-productinventory
production-productsubcategory
purchasing-purchaseorderdetail
purchasing-purchaseorderheader
purchasing-vendor
sales-creditcard
sales-customer
sales-salesorderdetail
sales-salesorderheader
sales-salesperson
sales-salesterritory
sales-store


# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [4]:
# Setup inicial do df para realizar a EDA 

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [5]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")


print("Todas as tabelas foram processadas com sucesso!")

Processando tabela: desafioadventureworks-446600.raw_data.sales-salesperson
Lendo os dados do BigQuery...
Tabela sales_salesperson processada e armazenada com sucesso.
Todas as tabelas foram processadas com sucesso!


In [6]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

Variável criada: sales_salesperson


In [7]:
# Atribuir o df a uma variável com nome mais simples
sales_salesperson = df_processados['sales_salesperson']

print(f"Colunas: {sales_salesperson.shape[1]}\nLinhas: {sales_salesperson.shape[0]}")

Colunas: 9
Linhas: 68


In [8]:
# Identificar duplicadas
duplicadas = sales_salesperson[sales_salesperson.duplicated(subset=['businessentityid'], keep=False)]

# Verificar se existem duplicadas
if not duplicadas.empty:
    
    duplicadas_ordenadas = duplicadas.sort_values(by=['businessentityid', 'modifieddate'])

    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

duplicadas ordenadas:
    businessentityid  territoryid  salesquota  bonus  commissionpct   salesytd  saleslastyear                               rowguid              modifieddate
0                274         <NA>        <NA>      0           0.00  559697.56           0.00  48754992-9ee0-4c0e-8c94-9451604e3e02 2010-12-28 00:00:00+00:00
1                274         <NA>        <NA>      0           0.00  559697.56           0.00  48754992-9ee0-4c0e-8c94-9451604e3e02 2010-12-28 00:00:00+00:00
2                274         <NA>        <NA>      0           0.00  559697.56           0.00  48754992-9ee0-4c0e-8c94-9451604e3e02 2010-12-28 00:00:00+00:00
3                274         <NA>        <NA>      0           0.00  559697.56           0.00  48754992-9ee0-4c0e-8c94-9451604e3e02 2010-12-28 00:00:00+00:00
56               275            2      300000   4100           0.01 3763178.18     1750406.48  1e0a7274-3064-4f58-88ee-4c6586c87169 2011-05-24 00:00:00+00:00
..               ...          

In [9]:
# Remover duplicadas* 
sales_salesperson = sales_salesperson.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(sales_salesperson)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = sales_salesperson.copy()


#doc: bkp = backup (cópia)
#doc*: mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
#      Importante, pois se houver erro na ingestão (duplicação), mantém os dados íntegros.

Linhas após remover duplicadas (baseando-se na última 'modifieddate'): 17


In [10]:
# Ordenar e exibir o df por 'businessentityid'
sales_salesperson = sales_salesperson.sort_values(by=['businessentityid'])

print(sales_salesperson)

    businessentityid  territoryid  salesquota  bonus  commissionpct   salesytd  saleslastyear                               rowguid              modifieddate
3                274         <NA>        <NA>      0           0.00  559697.56           0.00  48754992-9ee0-4c0e-8c94-9451604e3e02 2010-12-28 00:00:00+00:00
59               275            2      300000   4100           0.01 3763178.18     1750406.48  1e0a7274-3064-4f58-88ee-4c6586c87169 2011-05-24 00:00:00+00:00
15               276            4      250000   2000           0.01 4251368.55     1439156.03  4dd9eee4-8e81-4f8c-af97-683394c1f7c0 2011-05-24 00:00:00+00:00
19               277            3      250000   2500           0.01 3189418.37     1997186.20  39012928-bfec-4242-874d-423162c3f567 2011-05-24 00:00:00+00:00
23               278            6      250000    500           0.01 1453719.47     1620276.90  7a0ae1ab-b283-40f9-91d1-167abf06d720 2011-05-24 00:00:00+00:00
63               279            5      300000   6700

In [11]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in sales_salesperson.columns:   
    missing_rows = sales_salesperson[sales_salesperson[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

Coluna 'businessentityid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'businessentityid'.

Coluna 'territoryid': 3 linhas ausentes.
Exibindo as primeiras linhas com valores ausentes em 'territoryid':
    businessentityid  territoryid  salesquota  bonus  commissionpct  salesytd  saleslastyear                               rowguid              modifieddate
3                274         <NA>        <NA>      0           0.00 559697.56           0.00  48754992-9ee0-4c0e-8c94-9451604e3e02 2010-12-28 00:00:00+00:00
7                285         <NA>        <NA>      0           0.00 172524.45           0.00  cfdbef27-b1f7-4a56-a878-0221c73bae67 2013-03-07 00:00:00+00:00
11               287         <NA>        <NA>      0           0.00 519905.93           0.00  1dd1f689-df74-4149-8600-59555eef154b 2012-04-09 00:00:00+00:00 

Coluna 'salesquota': 3 linhas ausentes.
Exibindo as primeiras linhas com valores ausentes em 'salesquota':
    businessentityid  territoryid  salesquota  bo

In [12]:
# Substituir valores NA na colunas por 0
sales_salesperson['territoryid'] = sales_salesperson['territoryid'].fillna(0)
sales_salesperson['salesquota'] = sales_salesperson['salesquota'].fillna(0)

print(sales_salesperson)


#doc*: para fins de análise, optei em não apagar os registros, já que são poucos e importantes registros. (dados)

    businessentityid  territoryid  salesquota  bonus  commissionpct   salesytd  saleslastyear                               rowguid              modifieddate
3                274            0           0      0           0.00  559697.56           0.00  48754992-9ee0-4c0e-8c94-9451604e3e02 2010-12-28 00:00:00+00:00
59               275            2      300000   4100           0.01 3763178.18     1750406.48  1e0a7274-3064-4f58-88ee-4c6586c87169 2011-05-24 00:00:00+00:00
15               276            4      250000   2000           0.01 4251368.55     1439156.03  4dd9eee4-8e81-4f8c-af97-683394c1f7c0 2011-05-24 00:00:00+00:00
19               277            3      250000   2500           0.01 3189418.37     1997186.20  39012928-bfec-4242-874d-423162c3f567 2011-05-24 00:00:00+00:00
23               278            6      250000    500           0.01 1453719.47     1620276.90  7a0ae1ab-b283-40f9-91d1-167abf06d720 2011-05-24 00:00:00+00:00
63               279            5      300000   6700

In [13]:
#verificar informações do df
sales_salesperson.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, 3 to 55
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   businessentityid  17 non-null     Int64              
 1   territoryid       17 non-null     Int64              
 2   salesquota        17 non-null     Int64              
 3   bonus             17 non-null     Int64              
 4   commissionpct     17 non-null     float64            
 5   salesytd          17 non-null     float64            
 6   saleslastyear     17 non-null     float64            
 7   rowguid           17 non-null     object             
 8   modifieddate      17 non-null     datetime64[us, UTC]
dtypes: Int64(4), datetime64[us, UTC](1), float64(3), object(1)
memory usage: 1.4+ KB


In [14]:
# Variáveis quantitativas*: estatísticas descritivas para verificar se ainda há o que ser feito antes de exportar os dados ao BigQuery

# Identificar colunas numéricas para análise de outliers
numeric_columns = ['salesquota', 'bonus','commissionpct','salesytd','saleslastyear']

# Estatísticas Descritivas das colunas numéricas*
print(sales_salesperson[numeric_columns].describe())

# Cálculo de limites para outliers (IQR)**
for col in numeric_columns:
    q1 = sales_salesperson[col].quantile(0.25)
    q3 = sales_salesperson[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Detecção e Análise de Outliers***
    outliers = sales_salesperson[(sales_salesperson[col] < lower_bound) | (sales_salesperson[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


#doc*: variáveis quantitativas são um tipo de dado que pode ser representado por números e medidas objetivas
#doc*: realizar estatísticas descritivas para entender a centralidade e variação dos dados (valores médios, mínimos, máximos, etc.)
#doc**: calcular limites para identificar outliers (valores extremos que podem indicar erros ou casos excepcionais nos dados)
#doc***: verificar a existência de outliers para decidir ações como remoção, substituição ou tratamento, garantindo qualidade dos dados

#doc****: # Análise dos Resultados:

# Coluna: 
#       a maioria dos registros possui salesquota em 250.000, que é o valor esperado padrão.
#       os outliers encontrados são valores 0 (provavelmente representam vendedores sem cota definida) e valores 300.000 (que podem indicar cotas elevadas atribuídas a alguns vendedores).
#       coluna: commissionpct : os outliers detectados são valores iguais a 0, indicando que alguns vendedores não recebem comissão, o que pode ser normal.
#       coluna: saleslastyear: os outliers encontrados são valores 0, indicando que alguns vendedores não tiveram vendas no ano anterior, o que pode ser normal.

       salesquota   bonus  commissionpct   salesytd  saleslastyear
count       17.00   17.00          17.00      17.00          17.00
mean    214705.88 2859.41           0.01 2133975.99     1393291.98
std     104230.37 2273.31           0.01 1243721.37      849244.47
min          0.00    0.00           0.00  172524.45           0.00
25%     250000.00  500.00           0.01 1421810.92     1307949.79
50%     250000.00 3500.00           0.01 1827066.71     1635823.40
75%     250000.00 5000.00           0.02 3121616.32     1997186.20
max     300000.00 6700.00           0.02 4251368.55     2396539.76

Coluna: salesquota
Limite inferior: 250000.0, Limite superior: 250000.0
Outliers detectados (6):
    salesquota
3            0
59      300000
63      300000
67      300000
7            0
11           0

Coluna: bonus
Limite inferior: -6250.0, Limite superior: 11750.0
Outliers detectados (0):
Empty DataFrame
Columns: [bonus]
Index: []

Coluna: commissionpct
Limite inferior: 0.000999999999999999

In [15]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['sales_salesperson'] = sales_salesperson

In [16]:
sales_salesperson.head()

,businessentityid,territoryid,salesquota,bonus,commissionpct,salesytd,saleslastyear,rowguid,modifieddate
3,274,0,0,0,0.00,559697.56,0.00,48754992-9ee0-4c0e-8c94-9451604e3e02,2010-12-28 00:00:00+00:00
59,275,2,300000,4100,0.01,3763178.18,1750406.48,1e0a7274-3064-4f58-88ee-4c6586c87169,2011-05-24 00:00:00+00:00
15,276,4,250000,2000,0.01,4251368.55,1439156.03,4dd9eee4-8e81-4f8c-af97-683394c1f7c0,2011-05-24 00:00:00+00:00
19,277,3,250000,2500,0.01,3189418.37,1997186.20,39012928-bfec-4242-874d-423162c3f567,2011-05-24 00:00:00+00:00
23,278,6,250000,500,0.01,1453719.47,1620276.90,7a0ae1ab-b283-40f9-91d1-167abf06d720,2011-05-24 00:00:00+00:00


In [17]:
# Padronizar colunas com valores textuais

sales_salesperson['rowguid'] = sales_salesperson['rowguid'].str.strip().str.upper()

print(sales_salesperson.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

    businessentityid  territoryid  salesquota  bonus  commissionpct   salesytd  saleslastyear                               rowguid              modifieddate
3                274            0           0      0           0.00  559697.56           0.00  48754992-9EE0-4C0E-8C94-9451604E3E02 2010-12-28 00:00:00+00:00
59               275            2      300000   4100           0.01 3763178.18     1750406.48  1E0A7274-3064-4F58-88EE-4C6586C87169 2011-05-24 00:00:00+00:00
15               276            4      250000   2000           0.01 4251368.55     1439156.03  4DD9EEE4-8E81-4F8C-AF97-683394C1F7C0 2011-05-24 00:00:00+00:00
19               277            3      250000   2500           0.01 3189418.37     1997186.20  39012928-BFEC-4242-874D-423162C3F567 2011-05-24 00:00:00+00:00
23               278            6      250000    500           0.01 1453719.47     1620276.90  7A0AE1AB-B283-40F9-91D1-167ABF06D720 2011-05-24 00:00:00+00:00


In [18]:
# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
 
    output_table = f"{output_dataset}.{table_name}"
   
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")

Tabela sales_salesperson exportada com sucesso para desafioadventureworks-446600.raw_data_cleaned.sales_salesperson.
